In [23]:
import pandas as pd
import numpy as np
import random
from IPython.display import clear_output

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Conv1D, Conv2D, GlobalMaxPooling1D, MaxPooling2D, Flatten, Dense, Input, concatenate
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from keras.layers.embeddings import Embedding

from sklearn.metrics import recall_score, accuracy_score

import tensorflow_hub as hub

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
df1 = pd.read_csv('All_US_tweets.csv')

In [4]:
df1.head()

,Timestamp,TweetText,Closest_State,Closest_City,Region
0,10229,i gotta get you readded to bbm,Alabama,Adamsville,South
1,16439,ahhh yes,Alabama,Adamsville,South
2,17131,an old locksmith,Alabama,Adamsville,South
3,2648,rt they are shooting at pentagon metro please...,Alabama,Adamsville,South
4,2750,as a matter of fact i wanna ask about that,Alabama,Adamsville,South


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374519 entries, 0 to 374518
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Timestamp      374519 non-null  int64 
 1   TweetText      374294 non-null  object
 2   Closest_State  374519 non-null  object
 3   Closest_City   374519 non-null  object
 4   Region         374519 non-null  object
dtypes: int64(1), object(4)
memory usage: 14.3+ MB


In [6]:
# Max string length
df1["TweetText"].str.split(" ").str.len().describe(percentiles=[0.01, 0.5, 0.99])

count    374294.000000
mean         11.843054
std           7.576504
min           1.000000
1%            2.000000
50%          11.000000
99%          29.000000
max         710.000000
Name: TweetText, dtype: float64

In [7]:
max_length = 29

In [8]:
max_length = 29
df1['TweetText'] = df1['TweetText'].astype(str)
df1 = df1[df1['TweetText'].str.split(" ").str.len() <= max_length]
df1["TweetText"].str.split(" ").str.len().describe(percentiles=[0.01, 0.5, 0.99])

count    372389.000000
mean         11.709132
std           6.917127
min           1.000000
1%            2.000000
50%          10.000000
99%          28.000000
max          29.000000
Name: TweetText, dtype: float64

In [9]:
df2 = df1.copy()

In [10]:
df2.head()

,Timestamp,TweetText,Closest_State,Closest_City,Region
0,10229,i gotta get you readded to bbm,Alabama,Adamsville,South
1,16439,ahhh yes,Alabama,Adamsville,South
2,17131,an old locksmith,Alabama,Adamsville,South
3,2648,rt they are shooting at pentagon metro please...,Alabama,Adamsville,South
4,2750,as a matter of fact i wanna ask about that,Alabama,Adamsville,South


In [11]:
# one hot encoding
df3 = df2.copy()

one_hot = pd.get_dummies(df3['Region'])
df3 = df3.join(one_hot)
df3 = df3.drop('Region', axis = 1)
df3

,Timestamp,TweetText,Closest_State,Closest_City,Midwest,NonCont,Northeast,Rockies,South,Southwest,West Coast
0,10229,i gotta get you readded to bbm,Alabama,Adamsville,0,0,0,0,1,0,0
1,16439,ahhh yes,Alabama,Adamsville,0,0,0,0,1,0,0
2,17131,an old locksmith,Alabama,Adamsville,0,0,0,0,1,0,0
3,2648,rt they are shooting at pentagon metro please...,Alabama,Adamsville,0,0,0,0,1,0,0
4,2750,as a matter of fact i wanna ask about that,Alabama,Adamsville,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
374514,13789,lmao i can dig that fam imma def cop 09 tho lol,Wyoming,Rock Springs,0,0,0,1,0,0,0
374515,69985,ummmm ron artest wtfuck,Wyoming,Rock Springs,0,0,0,1,0,0,0
374516,79578,lol indeed gotta know how to play the game li...,Wyoming,Rock Springs,0,0,0,1,0,0,0
374517,79690,lmao my nerfherder u absolutely correct,Wyoming,Rock Springs,0,0,0,1,0,0,0


In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df3['TweetText'].tolist())

In [13]:
x_dat = df3[['TweetText']]
y_dat = df3[df2['Region'].unique()]

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x_dat, y_dat, test_size = 0.3, random_state = 50)

In [16]:
train_seq = tokenizer.texts_to_sequences(x_train['TweetText'].tolist())
train_seq = pad_sequences(train_seq, maxlen = max_length, padding = 'post', truncating = 'post')

In [17]:
embedding_dim = 150

In [29]:
model = Sequential()

model.add(Embedding(len(tokenizer.index_word) +1, embedding_dim, input_length = max_length))

model.add(Conv1D(20, kernel_size = 2, strides = 2, activation = 'relu'))
model.add(GlobalMaxPooling1D())

model.add(Dense(20, activation = 'relu'))
model.add(Dense(15, activation = 'relu'))
model.add(Dense(len(df2['Region'].unique()), activation = 'softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [32]:
model_history = model.fit(train_seq, y_train, epochs=5)

Epoch 1/5


ValueError: in user code:

    C:\Users\nicho\anaconda3\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\nicho\anaconda3\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\nicho\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\nicho\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\nicho\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\nicho\anaconda3\lib\site-packages\keras\engine\training.py:835 run_step  **
        outputs = model.train_step(data)
    C:\Users\n

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}'